In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [2]:
pc = initialize_pinecone()

In [3]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [4]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [5]:
input_song, input_artist = get_user_input()

In [6]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [7]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [8]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [9]:
input_data

,id,name,artists
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira


Similar songs

In [10]:
tracks_data = get_similar_songs(input_track_id, 5)

In [11]:
for track in tracks_data:
    print(track['name'])


Millones
Te Quiero Tanto, Tanto
El Amor
Aprender A Quererte
Mi Gente


In [12]:
tracks_data = pd.DataFrame(tracks_data)

In [13]:
tracks_data.head(2)

,id,name,artists
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo
1,2S0XH9R1BVZhbcy7j9yQbx,"Te Quiero Tanto, Tanto",Onda Vaselina


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [14]:
filter_existing_vector_id(index, tracks_data)            

In [15]:

tracks_data.reset_index(drop=True, inplace=True)

In [16]:
tracks_data

,id,name,artists
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino"""
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William"


In [17]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [18]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [19]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [20]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]"


In [21]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]"


Related tracks:

In [22]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, C, G, Am, F]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[F, G, Em, Am, F, G, C, Am]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[C, G, Am, F, C, G, F, C]"
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[Em, C, G, D, Em, C, G, D]"


In [23]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, F, C, G, Am]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[F, G, Em, Am, F, G, C, Am]","[C, G, Am, F, D, A, Bm, G]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[C, G, Am, F, C, G, F, C]","[G, D, Em, C, G, D, C, G]"
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[Em, C, G, D, Em, C, G, D]","[F#m, D, A, E, F#m, D, A, E]"


# API de Spotify para nuevas columnas

Input:

In [24]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0


In [25]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767


In [26]:
insert_lyrics_db(input_data)

(                       id                         name  artists  \
 0  3TY1PFZXmYeiLcXygB74My  Suerte (Whenever, Wherever)  Shakira   
 
                          chords                   instruments  speechiness  \
 0  [E, B, C#m, A, E, B, C#m, A]  [E, A, C#m, B, E, A, C#m, B]       0.0337   
 
    instrumentalness  key  mode    tempo  loudness  \
 0                 0    1     0  108.977    -4.767   
 
                                                theme  
 0  Suerte que en el sur hayas nacido\nY que burle...  ,
 'Suerte que en el sur hayas nacido\nY que burlemos las distancias,\nSuerte que es haberte conocido\nY por ti amar tierras extrañas\nYo puedo escalar los andes solo\nPor ir a contar tus lunares,\n\nContigo celebro y sufro todo')

In [27]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [28]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...


Related tracks:

In [29]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, F, C, G, Am]",0.2760,0.000000
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[F, G, Em, Am, F, G, C, Am]","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[C, G, Am, F, C, G, F, C]","[G, D, Em, C, G, D, C, G]",0.0334,0.000000
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[Em, C, G, D, Em, C, G, D]","[F#m, D, A, E, F#m, D, A, E]",0.0772,0.000024


In [30]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[F, G, Em, Am, F, G, C, Am]","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[C, G, Am, F, C, G, F, C]","[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[Em, C, G, D, Em, C, G, D]","[F#m, D, A, E, F#m, D, A, E]",0.0772,0.000024,11,0,104.666,-4.838


In [31]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para J Balvin, Willy William - Mi Gente


(                       id                 name                  artists  \
 0  3YAnNBdk0uHyakXXzMbNJq             Millones                   Camilo   
 1  4jUivlRr5KY3BaSc3vwNA8              El Amor        Tito "El Bambino"   
 2  3zNcn4BaVfKORyx3uDfruW  Aprender A Quererte                    Morat   
 3  7COfe3P7KgfwDwIRB8LIDw             Mi Gente  J Balvin, Willy William   
 
                         chords                   instruments  speechiness  \
 0   [C, G, Am, F, C, G, Am, F]    [C, G, Am, F, F, C, G, Am]       0.2760   
 1  [F, G, Em, Am, F, G, C, Am]    [C, G, Am, F, D, A, Bm, G]       0.0495   
 2    [C, G, Am, F, C, G, F, C]     [G, D, Em, C, G, D, C, G]       0.0334   
 3   [Em, C, G, D, Em, C, G, D]  [F#m, D, A, E, F#m, D, A, E]       0.0772   
 
    instrumentalness  key  mode    tempo  loudness  \
 0          0.000000   10     1  190.087    -2.312   
 1          0.000014   11     1  103.011    -7.918   
 2          0.000000    8     1  101.013    -5.191   
 3        

In [32]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

Input:

In [33]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...


In [34]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [35]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[E, B, C#m, A]","[E, B, C#m, A]"


In [36]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [37]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[E, B, C#m, A]","[E, B, C#m, A]"


In [38]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [39]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]"


In [40]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]","[3, 10, 12, 8]","[3, 10, 12, 8]"


In [41]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]","[3, 10, 12, 8]","[3, 10, 12, 8]","[7, 2, -4]","[7, 2, -4]"


In [42]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [43]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]","[3, 10, 12, 8]","[3, 10, 12, 8]","[7, 2, -4]","[7, 2, -4]"


Related tracks:

In [44]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [45]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera\nPorque qué bien te que...,"[C, G, Am, F]","[C, G, Am, F]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia\nUna simple fantasia\nEs ...,"[F, G, Em, Am]","[F, G, C, Am]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro \nUna ...,"[C, G, Am, F]","[C, G, F, C]"


In [46]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [47]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera\nPorque qué bien te que...,"[C, G, Am, F]","[C, G, Am, F]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia\nUna simple fantasia\nEs ...,"[F, G, Em, Am]","[F, G, C, Am]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro \nUna ...,"[C, G, Am, F]","[C, G, F, C]"


In [48]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [49]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera\nPorque qué bien te que...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia\nUna simple fantasia\nEs ...,"[F, G, Em, Am]","[F, G, C, Am]","[5, 7, 16, 21]","[5, 7, 0, 21]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro \nUna ...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]"


In [50]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera\nPorque qué bien te que...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-10, -3, 11, -5]","[-10, -3, 11, -5]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia\nUna simple fantasia\nEs ...,"[F, G, Em, Am]","[F, G, C, Am]","[5, 7, 16, 21]","[5, 7, 0, 21]","[-6, -4, 5, 10]","[-6, -4, -11, 10]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro \nUna ...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-8, -1, 13, -3]","[-8, -1, -3, -8]"
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[F#m, D, A, E, F#m, D, A, E]",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...","[Em, C, G, D]","[Em, C, G, D]","[16, 0, 7, 2]","[16, 0, 7, 2]","[5, -11, -4, -9]","[5, -11, -4, -9]"


In [51]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera\nPorque qué bien te que...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-10, -3, 11, -5]","[-10, -3, 11, -5]","[7, 14, -16]","[7, 14, -16]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia\nUna simple fantasia\nEs ...,"[F, G, Em, Am]","[F, G, C, Am]","[5, 7, 16, 21]","[5, 7, 0, 21]","[-6, -4, 5, 10]","[-6, -4, -11, 10]","[2, 9, 5]","[2, -7, 21]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro \nUna ...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-8, -1, 13, -3]","[-8, -1, -3, -8]","[7, 14, -16]","[7, -2, -5]"
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William","[F#m, D, A, E, F#m, D, A, E]",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...","[Em, C, G, D]","[Em, C, G, D]","[16, 0, 7, 2]","[16, 0, 7, 2]","[5, -11, -4, -9]","[5, -11, -4, -9]","[-16, 7, -5]","[-16, 7, -5]"


In [52]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [53]:
df_vectorial = df_vectorial.fillna(0)

In [54]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,"[C, G, Am, F, F, C, G, Am]",0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-10, -3, 11, -5]","[-10, -3, 11, -5]","[7, 14, -16]","[7, 14, -16]"
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""","[C, G, Am, F, D, A, Bm, G]",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,"[F, G, Em, Am]","[F, G, C, Am]","[5, 7, 16, 21]","[5, 7, 0, 21]","[-6, -4, 5, 10]","[-6, -4, -11, 10]","[2, 9, 5]","[2, -7, 21]"
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,"[G, D, Em, C, G, D, C, G]",0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-8, -1, 13, -3]","[-8, -1, -3, -8]","[7, 14, -16]","[7, -2, -5]"


In [55]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]","[3, 10, 12, 8]","[3, 10, 12, 8]","[7, 2, -4]","[7, 2, -4]"


# Backup csv

In [56]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:277: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:283: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [57]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [58]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]","[E, A, C#m, B, E, A, C#m, B]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido Y que burlem...,"[E, B, C#m, A]","[E, B, C#m, A]","[4, 11, 13, 9]","[4, 11, 13, 9]","[3, 10, 12, 8]","[3, 10, 12, 8]","[7, 2, -4]","[7, 2, -4]"


In [59]:
columns_to_expand

['instruments',
 'chords_1',
 'chords_2',
 'chords_1_numeric',
 'chords_2_numeric',
 'normal_chords_1_numeric',
 'normal_chords_2_numeric',
 'diff_chords_1_numeric',
 'diff_chords_2_numeric']

In [60]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [61]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[E, B, C#m, A, E, B, C#m, A]",0.0337,0,1,0,108.977,-4.767,...,3,10,12,8,7,2,-4,7,2,-4


In [62]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [63]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [64]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,-10,-3,11,-5,7,14,-16,7,14,-16
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-6,-4,-11,10,2,9,5,2,-7,21
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-8,-1,-3,-8,7,14,-16,7,-2,-5
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...",...,5,-11,-4,-9,-16,7,-5,-16,7,-5


In [65]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [66]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,-10,-3,11,-5,7,14,-16,7,14,-16
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-6,-4,-11,10,2,9,5,2,-7,21
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-8,-1,-3,-8,7,14,-16,7,-2,-5
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...",...,5,-11,-4,-9,-16,7,-5,-16,7,-5


In [67]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,-10,-3,11,-5,7,14,-16,7,14,-16
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-6,-4,-11,10,2,9,5,2,-7,21
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-8,-1,-3,-8,7,14,-16,7,-2,-5


# Create column for the text to create the embeddings

Input:

In [68]:
input_expanded=to_embbed_text_column(input_expanded)

In [69]:
input_expanded['to_embbed_text']

0    name: Suerte (Whenever, Wherever) artists: Sha...
Name: to_embbed_text, dtype: object

Related songs:

In [70]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [71]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,-3,11,-5,7,14,-16,7,14,-16,name: Millones artists: Camilo speechiness: 0....
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-4,-11,10,2,9,5,2,-7,21,"name: El Amor artists: Tito ""El Bambino"" speec..."
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-1,-3,-8,7,14,-16,7,-2,-5,name: Aprender A Quererte artists: Morat speec...
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...",...,-11,-4,-9,-16,7,-5,-16,7,-5,"name: Mi Gente artists: J Balvin, Willy Willia..."


# Text for embeddings to chat gpt

In [72]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [73]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [74]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [75]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [76]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,-3,11,-5,7,14,-16,7,14,-16,name: Millones artists: Camilo speechiness: 0....
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-4,-11,10,2,9,5,2,-7,21,"name: El Amor artists: Tito ""El Bambino"" speec..."
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-1,-3,-8,7,14,-16,7,-2,-5,name: Aprender A Quererte artists: Morat speec...
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...",...,-11,-4,-9,-16,7,-5,-16,7,-5,"name: Mi Gente artists: J Balvin, Willy Willia..."


Related songs:

In [77]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [78]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [79]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [80]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,3YAnNBdk0uHyakXXzMbNJq,Millones,Camilo,0.2760,0.000000,10,1,190.087,-2.312,Tú debes ser ingeniera Porque qué bien te qued...,...,11,-5,7,14,-16,7,14,-16,"[0.026146424934267998, 0.022650258615612984, 0...","{'name': 'Millones', 'artists': 'Camilo', 'spe..."
1,4jUivlRr5KY3BaSc3vwNA8,El Amor,"Tito ""El Bambino""",0.0495,0.000014,11,1,103.011,-7.918,El amor es una magia Una simple fantasia Es co...,...,-11,10,2,9,5,2,-7,21,"[0.00954257883131504, 0.010037379339337349, -0...","{'name': 'El Amor', 'artists': 'Tito ""El Bambi..."
2,3zNcn4BaVfKORyx3uDfruW,Aprender A Quererte,Morat,0.0334,0.000000,8,1,101.013,-5.191,Cuando te vi senti algo raro por dentro Una m...,...,-3,-8,7,14,-16,7,-2,-5,"[0.018813414499163628, 0.014904949814081192, -...","{'name': 'Aprender A Quererte', 'artists': 'Mo..."
3,7COfe3P7KgfwDwIRB8LIDw,Mi Gente,"J Balvin, Willy William",0.0772,0.000024,11,0,104.666,-4.838,"""Mi Gente"" by J Balvin and Willy William celeb...",...,-4,-9,-16,7,-5,-16,7,-5,"[0.023200582712888718, -0.002065642038360238, ...","{'name': 'Mi Gente', 'artists': 'J Balvin, Wil..."


In [82]:
df_para_pinecone

,id,values,metadata
0,3YAnNBdk0uHyakXXzMbNJq,"[0.026146424934267998, 0.022650258615612984, 0...","{'name': 'Millones', 'artists': 'Camilo', 'spe..."
1,4jUivlRr5KY3BaSc3vwNA8,"[0.00954257883131504, 0.010037379339337349, -0...","{'name': 'El Amor', 'artists': 'Tito ""El Bambi..."
2,3zNcn4BaVfKORyx3uDfruW,"[0.018813414499163628, 0.014904949814081192, -...","{'name': 'Aprender A Quererte', 'artists': 'Mo..."
3,7COfe3P7KgfwDwIRB8LIDw,"[0.023200582712888718, -0.002065642038360238, ...","{'name': 'Mi Gente', 'artists': 'J Balvin, Wil..."


# Query search in pinecone

In [83]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [84]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [85]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: piano guitar, Tempo: 114.032, Speechiness: 0.0388, Loudness: -6.153',
 'Intruments: guitar drums, Tempo: 117.252, Speechiness: 0.07, Loudness: -7.16']

In [86]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.199061543
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.19640702
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.191190347
Letra: The theme of "Miles and Miles of Texas" revolves around the deep connection to the vast Texas landscape and the sense of freedom it brings to the narrator. It reflects a longing for home and the cherished memories tied to the plac

In [87]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [88]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [89]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.452431619
Chords: C G Am F, D A Bm G, 
 Score: 0.448184758
Chords: C G Am F, C F G Am, 
 Score: 0.438908577
Chords: C E7 F C, C E7 F C, 
 Score: 0.436558366
Chords: C G Am F, C G F F, 
 Score: 0.433311701


In [90]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [91]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, D A Bm G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C E7 F C, C E7 F C',
 'Chord wheel: C G Am F, C G F F']

In [92]:
input_data['theme'][0]

'Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo'

In [93]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [94]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo 3TY1PFZXmYeiLcXygB74My Score: 0.543976963
Lyrics: Dejare mi tierra por ti, Dejare mis campos y me iré Lejos de aquí, Cruzaré llorando el jardín y con tus recuerdos partiré, lejos de aquí  De día viviré pensando en tus sonrisas  5IU4JO2cpa0wPK8oQWFfCr Score: 0.486122221
Lyrics: Te conozco de otra vida, hoy saldrás por la ventana Como un puñal, rasgando las tinieblas si te llevo es para que me lleves, au oh Sin hablar, solo roces  quiero ver tus ojos rojos 5rGphVLnbPdyBQ5X3rjb2X Score: 0.48132807
Lyrics: El me dejo y yo queria hacerlo Maldita sea Esa soy yo La que aguanta patanes por ser tan buena 1JAv5OTUlOFgA2XPmA5vzd Score: 0.479669064
Lyrics: Ahora que tenemos tiempo  enfadarse no vale la pena,  pues entre días de guerra hay paz  entiendo que tengamos problemas 

In [95]:
lyrics_list

['Lyrics: Suerte que en el sur hayas nacido Y que burlemos las distancias, Suerte que es haberte conocido Y por ti amar tierras extrañas Yo puedo escalar los andes solo Por ir a contar tus lunares,  Contigo celebro y sufro todo',
 'Lyrics: Dejare mi tierra por ti, Dejare mis campos y me iré Lejos de aquí, Cruzaré llorando el jardín y con tus recuerdos partiré, lejos de aquí  De día viviré pensando en tus sonrisas ',
 'Lyrics: Te conozco de otra vida, hoy saldrás por la ventana Como un puñal, rasgando las tinieblas si te llevo es para que me lleves, au oh Sin hablar, solo roces  quiero ver tus ojos rojos',
 'Lyrics: El me dejo y yo queria hacerlo Maldita sea Esa soy yo La que aguanta patanes por ser tan buena',
 'Lyrics: Ahora que tenemos tiempo\xa0 enfadarse no vale la pena,\xa0 pues entre días de guerra hay paz\xa0 entiendo que tengamos problemas y el miedo no deja empatizar.\xa0 Y si te sientes raro coge mi mano\xa0 y estarás acompañado. Ven a celebrarlo.\xa0  Contentos con lo poco q

In [96]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

# Título de la canción: Distancias y Cercanías

**Tempo:** 131 BPM  
**Instrumentos principales:** Guitarra, Bajo

### Letra y acordes:

1. **Verso 1:**

   ```
   [C] Suerte que existes, amor a [G] mi lado,  
   [Am] En tierras lejanas mi corazón [F] ha dejado,  
   [F] Puente entre dos mundos, el destino [Am] me ha mostrado,  
   [C] Contigo celebro y sufro, pero [G] nunca he fracasado.  
   ```

2. **Coro:**

   ```
   [C] Distancias que cruzo, [G] solo por verte,  
   [Am] En tus lunares hallo razones [F] para quererte,  
   [F] Mis días de guerra se vuelven [Am] calmados,  
   [C] Con tu abrazo efímero, [G] siempre renovados.  
   ```

3. **Verso 2:**

   ```
   [C] Dejo atrás mi tierra, mi [G] vida pasada,  
   [Am] Riéndome con tus sueños, la [F] tristeza fue menguada,  
   [F] Escalando cumbres, mi [Am] espíritu se halla,  
   [C] A tu lado, mi mundo, [G] siempre estalla.  
   ```

4. **Coro final:**

   ```
   [C] Distancias burladas, [G] por siempre a tu lado,  
   [Am] En nu

In [99]:
nueva_cancion

'# Título de la canción: Distancias y Cercanías\n\n**Tempo:** 131 BPM  \n**Instrumentos principales:** Guitarra, Bajo\n\n### Letra y acordes:\n\n1. **Verso 1:**\n\n   ```\n   [C] Suerte que existes, amor a [G] mi lado,  \n   [Am] En tierras lejanas mi corazón [F] ha dejado,  \n   [F] Puente entre dos mundos, el destino [Am] me ha mostrado,  \n   [C] Contigo celebro y sufro, pero [G] nunca he fracasado.  \n   ```\n\n2. **Coro:**\n\n   ```\n   [C] Distancias que cruzo, [G] solo por verte,  \n   [Am] En tus lunares hallo razones [F] para quererte,  \n   [F] Mis días de guerra se vuelven [Am] calmados,  \n   [C] Con tu abrazo efímero, [G] siempre renovados.  \n   ```\n\n3. **Verso 2:**\n\n   ```\n   [C] Dejo atrás mi tierra, mi [G] vida pasada,  \n   [Am] Riéndome con tus sueños, la [F] tristeza fue menguada,  \n   [F] Escalando cumbres, mi [Am] espíritu se halla,  \n   [C] A tu lado, mi mundo, [G] siempre estalla.  \n   ```\n\n4. **Coro final:**\n\n   ```\n   [C] Distancias burladas, [G] 

AttributeError: 'str' object has no attribute 'choices'